<a href="https://colab.research.google.com/github/falcaodf/TRANSFER-LEARNING-YOLO/blob/main/YOLOV5_TRANSFER_LEARNING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Clonar repositório YOLOv5
!git clone https://github.com/ultralytics/yolov5
%cd yolov5

# Instalar dependências
!pip install -r requirements.txt



Cloning into 'yolov5'...
remote: Enumerating objects: 17564, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 17564 (delta 35), reused 7 (delta 7), pack-reused 17508 (from 3)
Receiving objects: 100% (17564/17564), 16.64 MiB | 30.15 MiB/s, done.
Resolving deltas: 100% (12029/12029), done.
/content/yolov5/yolov5/yolov5


In [ ]:
# Criar pastas
!mkdir -p /content/coco2017/images/val
!mkdir -p /content/coco2017/annotations

# Baixar imagens val2017
!wget -nc http://images.cocodataset.org/zips/val2017.zip
!unzip -o -q val2017.zip -d /content/coco2017/images/val  # -o para sobrescrever sem perguntar

# Baixar annotations
!wget -nc http://images.cocodataset.org/annotations/annotations_trainval2017.zip
!unzip -o -q annotations_trainval2017.zip -d /content/coco2017/annotations


--2025-09-04 23:50:00--  http://images.cocodataset.org/zips/val2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 3.5.25.117, 16.15.179.205, 54.231.234.193, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|3.5.25.117|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 815585330 (778M) [application/zip]
Saving to: ‘val2017.zip’

val2017.zip         100%[===================>] 777.80M  42.5MB/s    in 19s     

2025-09-04 23:50:19 (41.6 MB/s) - ‘val2017.zip’ saved [815585330/815585330]

--2025-09-04 23:50:28--  http://images.cocodataset.org/annotations/annotations_trainval2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.217.206.185, 52.216.208.105, 3.5.10.180, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|52.217.206.185|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252907541 (241M) [application/zip]
Saving to: ‘annotations_trainval2017.zip’

annotations_trainva 10

In [ ]:
import json
import os
from collections import defaultdict

# Caminhos
ann_file = '/content/coco2017/annotations/annotations/instances_val2017.json'
images_dir = '/content/coco2017/images/val'
labels_dir = '/content/coco2017/labels/val'

# Criar pasta de labels se não existir
os.makedirs(labels_dir, exist_ok=True)

# Ler JSON
with open(ann_file) as f:
    data = json.load(f)

# Mapear categorias
cat_map = {cat['id']: i for i, cat in enumerate(data['categories'])}

# Agrupar annotations por imagem
img_ann = defaultdict(list)
for ann in data['annotations']:
    img_ann[ann['image_id']].append(ann)

# Mapear imagem id → nome arquivo
img_id_to_name = {img['id']: img['file_name'] for img in data['images']}

# Converter cada annotation para YOLO
for img_id, anns in img_ann.items():
    file_name = img_id_to_name[img_id]
    # Encontrar altura e largura da imagem
    img_info = next(img for img in data['images'] if img['id'] == img_id)
    h, w = img_info['height'], img_info['width']
    txt_path = os.path.join(labels_dir, file_name.replace('.jpg', '.txt'))
    with open(txt_path, 'w') as f:
        for ann in anns:
            cat_id = cat_map[ann['category_id']]
            x, y, width, height = ann['bbox']
            x_center = (x + width / 2) / w
            y_center = (y + height / 2) / h
            w_norm = width / w
            h_norm = height / h
            f.write(f"{cat_id} {x_center} {y_center} {w_norm} {h_norm}\n")


In [ ]:
images_dir = '/content/coco2017/images/val'
labels_dir = '/content/coco2017/labels/val'
os.makedirs(labels_dir, exist_ok=True)




In [ ]:
# Criar arquivo coco_val.yaml apenas com val
yaml_content = """
train: /content/coco2017/images/val  # usar val como treino para teste rápido
val: /content/coco2017/images/val

nc: 80
names: [ 'person','bicycle','car','motorcycle','airplane','bus','train','truck','boat','traffic light',
         'fire hydrant','stop sign','parking meter','bench','bird','cat','dog','horse','sheep','cow',
         'elephant','bear','zebra','giraffe','backpack','umbrella','handbag','tie','suitcase','frisbee',
         'skis','snowboard','sports ball','kite','baseball bat','baseball glove','skateboard','surfboard',
         'tennis racket','bottle','wine glass','cup','fork','knife','spoon','bowl','banana','apple',
         'sandwich','orange','broccoli','carrot','hot dog','pizza','donut','cake','chair','couch',
         'potted plant','bed','dining table','toilet','tv','laptop','mouse','remote','keyboard','cell phone',
         'microwave','oven','toaster','sink','refrigerator','book','clock','vase','scissors','teddy bear',
         'hair drier','toothbrush' ]
"""

with open("/content/coco2017/coco_val.yaml", "w") as f:
    f.write(yaml_content)


In [ ]:
import os
os.environ["WANDB_MODE"] = "disabled"


In [ ]:
!python train.py --img 640 --batch 16 --epochs 50 \
--data /content/coco2017/coco_val.yaml --weights yolov5s.pt \
--name coco_val_model --device 0 --nosave


A saída de streaming foi truncada nas últimas 5000 linhas.
      37/49      6.09G    0.04359    0.05899     0.0143        192        640:  92% 289/313 [02:03<00:09,  2.61it/s]/content/yolov5/yolov5/yolov5/train.py:414: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
      37/49      6.09G    0.04359      0.059    0.01429        257        640:  93% 290/313 [02:04<00:09,  2.35it/s]/content/yolov5/yolov5/yolov5/train.py:414: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
      37/49      6.09G    0.04359    0.05898    0.01429        190        640:  93% 291/313 [02:04<00:08,  2.69it/s]/content/yolov5/yolov5/yolov5/train.py:414: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.

In [ ]:
!python detect.py --weights runs/train/coco_val_model/weights/best.pt \
--img 640 --conf 0.25 --source /content/coco2017/images/val


python3: can't open file '/content/detect.py': [Errno 2] No such file or directory
